### referenced by - https://github.com/kuleshov/audio-super-res
* 아래 audio-super-res로 명시된것은 위의 참고자료 모델을 의미함

## 1. VCTK데이터셋 다운로드 및 준비

* 학습에 사용될 VCTK데이터를 다운로드하고 압축을 푸는 과정을 수행한다.  
* 만약 다른 데이터셋을 사용하거나 미리 만든 데이터셋을 사용할 경우 이 단계는 수행 할 필요없다.  
* 단, 해당 폴더에 VCTK압축 파일을 저장해 두거나 별도의 경로를 Makefile에 지정해야함  
<blockquote>
<p>./audio-super-res/data/make</p>
</blockquote>

## 2. 학습데이터 파일 리스트 준비 및 .h5파일로 변환
### A. audio-super-res의 preprocess단계를 사용할 경우
* wav 음성 데이터 파일들의 (경로+파일이름)으로 구성된 텍스트파일을 준비한다  
(ex)./audio-super-res/data/vctk/multispeaker/train-files.txt
* wav 파일의 원본을 High-resolution, 학습용으로 preprocess된 파일을 Low-resolution 구성
* 위의 과정을 위해 기존 프로젝트에서는 single speaker와 multi speaker로 데이터셋을 분리하여 prep_vctk.py로 전처리했다
<blockquote>
<p>./audio-super-res/data/vctk/speaker1/make</p>
<p>./audio-super-res/data/vctk/multispeaker/make</p>
</blockquote>
* 위의 make작업으로 학습데이터 리스트를 바탕으로 prep_vctk.py에서 원본 데이터를 Low-Resolution으로 변환 후 잘게 자른 다음 h5파일로 구성하여 저장한다 
* 해당 방법의 자세한 내용은 상단 reference 페이지를 방문

### B. 별도의 전처리된 .wav파일을 사용할 경우
* input파일과 output파일을 준비해 ./data/input 과 ./data/output에 저장한다
* input파일 목록과 output파일 목록을 준비한다
* 아래와 같이 .wav파일들을 .h5파일로 변환한다

In [31]:
import os, argparse
import numpy as np
import h5py
import librosa
from scipy import interpolate
from scipy.signal import decimate
# ----------------------------------------------------------------------------'
args = {
    'out' : 'prep_audioset.h5',
    'out_val':'prep_audioset.val.h5',
    'input_file_list' : './data/inputfiles.txt', 
    'input_val_file_list':'./data/inputfiles.val.txt',
    'output_file_list': './data/outputfiles.txt',
    'output_val_file_list':'./data/outputfiles.val.txt',
    'in_dir': './data',
    'interpolate':1,
    'dimension':8192,
    'stride':3200, # 4096
    'scale':2, # 4
    'batch_size':1,
    'sr':16000,
    'sam':1.
}
# ----------------------------------------------------------------------------

In [32]:
from scipy.signal import butter, lfilter

def file2List(files):
    # add .txt file values in list array
    file_list = []
    file_extensions = set(['.wav'])
    with open(files) as f:
        for line in f:
            filename = line.strip()
            ext = os.path.splitext(filename)[1]
            if ext in file_extensions:
                file_list.append(os.path.join(args['in_dir'], filename))
    num_files = len(file_list)
    return file_list, num_files


def get_patch_data(file_list, num_files,args,d,s):
    patches = list()
    for j, file_path in enumerate(file_list):
        if j % 10 == 0: print('%d/%d' % (j, num_files))
        
        # load audio file
        x, fs = librosa.load(file_path, sr=args['sr']) # sr = sample rates
        
        # crop so that it works with scailing ratio
        x_len = len(x)
        x = x[ : x_len - (x_len % args['scale'])]
        
        # Do not generate low-res version. we already have them
        
        # Generate patches
        max_i = len(x) - d + 1 # d = dimension
        for i in range(0, max_i, s): # s = strides 
            # keep only a fraction of all the patches
            u = np.random.uniform()
            if u > args['sam']: continue
                        
            patch = np.array( x[i : i+d] )
            assert len(patch) == d

            # print _patch
            patches.append(patch.reshape((d,1)))
            
    return patches



In [33]:
def add_data_preprocessed(h5_file, inputfiles, outputfiles , args, save_examples=False):
    # Make a list of all preprocessed files
    input_file_list, num_input_files = file2List(inputfiles)
    output_file_list, num_output_files = file2List(outputfiles)

    
    # patches to extract and their size # lr not used
    if args['interpolate']:
        d, d_lr = args['dimension'], args['dimension']
        s, s_lr = args['stride'], args['stride']
    else:
        print('not interpolate')
        d, d_lr = args['dimension'], (int)(args['dimension'] / args['scale'])
        s, s_lr = args['stride'], (int)(args['stride'] / args['scale'])
        
        
    # get patches
    input_patches = get_patch_data(input_file_list, num_input_files,args,d_lr,s_lr)
    output_patches = get_patch_data(output_file_list, num_output_files,args,d,s)
    
    # crop # of patches so that it's a multiple of mini-batch size
    num_input_patches = len(input_patches)
    num_output_patches = len(output_patches)
    
    print('num_input_patches:', num_input_patches)
    print('num_output_patches:', num_output_patches)
    print('batch_size:', args['batch_size'])
    num_input_to_keep = int(np.floor(num_input_patches / args['batch_size']) * args['batch_size'])
    input_patches = np.array(input_patches[:num_input_to_keep])
    
    num_output_to_keep = int(np.floor(num_output_patches / args['batch_size']) * args['batch_size'])
    output_patches = np.array(output_patches[:num_output_to_keep])

    if save_examples:
        librosa.output.write_wav('example-hr.wav', output_patches[40][0], fs, norm=False)
        librosa.output.write_wav('example-lr.wav', input_patches[40][0], fs / args['scale'], norm=False)
        print (output_patches[40].shape)
        print (input_patches[40].shape)
        print (output_patches[40][0][:10])
        print (input_patches[40][0][:10])
        print ('two examples saved')

    print ('input_patches shape:',input_patches.shape)
    print ('output_patches shape:',output_patches.shape)

    # create the hdf5 file
    data_set = h5_file.create_dataset('data', input_patches.shape, np.float32) # lr
    label_set = h5_file.create_dataset('label', output_patches.shape, np.float32) # hr

    data_set[...] = input_patches
    label_set[...] = output_patches

In [34]:
if __name__ == '__main__':
    # create train
    with h5py.File(args['out'], 'w') as f:
        add_data_preprocessed(f, args['input_file_list'],args['output_file_list'], args, save_examples=False)

0/1
0/1
num_input_patches: 48
num_output_patches: 48
batch_size: 1
input_patches shape: (48, 8192, 1)
output_patches shape: (48, 8192, 1)


## 3. 마찬가지로 평가 데이터셋도 h5파일로 구성한다
* 위의 B경우에 해당

In [35]:
    # create train
    with h5py.File(args['out_val'], 'w') as f:
        add_data_preprocessed(f, args['input_val_file_list'],args['output_val_file_list'], args, save_examples=False)

0/1
0/1
num_input_patches: 48
num_output_patches: 48
batch_size: 1
input_patches shape: (48, 8192, 1)
output_patches shape: (48, 8192, 1)


## 4. 데이터셋의 정보를 시각화하여 확인
* 입, 출력 데이터가 올바르게 들어갔는지 확인하기 위함